In this document, we quantify the amount of cross-talk occuring between pixels in a CMOS image sensor. We believe that one cause of this cross-talk is diffusion of minority carriers (electrons): when a photon is absorbed, creating a photoelectron, close to the surface of a backside-illuminated sensor (outside of the depletion region), the electron may diffuse into nearby pixels.

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [3]:
%matplotlib qt
data_folder = os.getcwd() + '/data/IPRF Data/Spot Characterization/CMS'
# data_folder = os.getcwd() + '/data/IPRF Data/Spot Characterization/IMX455'
wavelengths = np.array([400, 625, 875])

for level in ['faint', 'medium', 'bright']:
    for ind in ['1', '2', '3']:
        filename = data_folder + '/NIR/spot_image_' + level + '_' + ind + '.fits'
        test_data = fits.open(filename)
        # test_data = fits.open(data_folder + '/IMX455_spot_1.fits')
        # Extract data from fits file
        test_data = test_data[0].data
        dark_val = np.mean(test_data[-2:])
        test_data = test_data[10:20, 10:20]
        # plt.imshow(test_data, cmap='gray')
        # plt.show()
        print(level + ' ' + ind)
        print('Dark Value:', dark_val)
        print('Central Pixel Value:', np.max(test_data))
        print('Central Pixel Fraction:', (np.max(test_data) - dark_val) / (np.sum(test_data) - dark_val * test_data.size))
        # print(np.sum(brightest - dark_val) / (np.sum(test_data) - dark_val * test_data.size))
        print()


faint 1
Dark Value: 0.46774193548387094
Central Pixel Value: 2959
Central Pixel Fraction: 0.604247511249613

faint 2
Dark Value: -0.8870967741935484
Central Pixel Value: 2865
Central Pixel Fraction: 0.5627430735903315

faint 3
Dark Value: 0.3387096774193548
Central Pixel Value: 2924
Central Pixel Fraction: 0.5782410361107566

medium 1
Dark Value: 0.8870967741935484
Central Pixel Value: 17406
Central Pixel Fraction: 0.5254962211227551

medium 2
Dark Value: -2.6774193548387095
Central Pixel Value: 16897
Central Pixel Fraction: 0.5089230811077035

medium 3
Dark Value: 0.03225806451612903
Central Pixel Value: 16663
Central Pixel Fraction: 0.5117497404357509

bright 1
Dark Value: 0.2903225806451613
Central Pixel Value: 36046
Central Pixel Fraction: 0.4897450506874471

bright 2
Dark Value: 3.4193548387096775
Central Pixel Value: 36380
Central Pixel Fraction: 0.4951345987503897

bright 3
Dark Value: 1.5483870967741935
Central Pixel Value: 35557
Central Pixel Fraction: 0.49107595551432887



Simulate how electron diffusion can cause crosstalk

In [144]:
global lost_electrons, collected_electrons, electron_positions

# Do some simulations
n_photons = 5000
abs_length = 3 # microns
depletion_depth = 5 # microns
sensor_depth = 10 # microns
sensor_width = 10 # microns
z_bins = np.linspace(0, sensor_depth, 100)
# Calculate number of electrons that interact within each bin
n_electrons_binned = n_photons * (np.exp(-z_bins / abs_length))
n_electrons_binned[1:] = n_electrons_binned[:-1] - n_electrons_binned[1:]
n_electrons_binned[0] = n_electrons_binned[0] - n_photons
n_electrons_binned = np.rint(n_electrons_binned).astype(int)
num_electrons = np.sum(n_electrons_binned)
# Initialize an array where each element corresponds to an electron's position
electron_positions = np.zeros((num_electrons, 3))
# The first 2 columns are the x and y positions of the electron, which starts at (0,0)
# as the spot is centered on a pixel. The z position is the depth of the electron,
# initialized by the depth at which it was created.
electron_positions[:,2] = np.repeat(z_bins, n_electrons_binned)
# Array that will accumulate the electrons that are collected in each pixel.
collected_electrons = np.zeros((11, 11))
def collect_electrons():
    global collected_electrons, electron_positions
    # The electron is collected if it reaches the depletion depth.
    # If this is the case, set their z position to NaN to indicate they were collected.
    # If this is the case, use the x and y positions to determine the pixel
    # the electron was collected in.
    collected = np.where(electron_positions[:,2] >= sensor_depth - depletion_depth)
    electron_positions[collected,2] = np.NaN
    pix_locs = np.rint(electron_positions[collected,:2] / sensor_width).astype(int)[0]
    # Shift so that center pixel corresponds goes to center of collected_electrons array
    pix_locs = pix_locs + [5, 5]
    for loc in pix_locs:
        collected_electrons[loc[0], loc[1]] += 1

lost_electrons = 0
def lose_electrons(loss_probability=0.01):
    global lost_electrons, electron_positions
    # The electron may be lost if it reaches the sensor surface. Otherwise
    # it may keep diffusing around.
    # Any electron can also be lost due to recombination. But the carrier lifetime is
    # O(100 us) so we can ignore this when we're on scales of O(100 ns).
    # If an electron is lost, set its z position to NaN.
    at_surface = np.where(electron_positions[:,2] < 0)
    lost = np.random.uniform(0, 1, len(at_surface[0])) < loss_probability
    actually_lost = at_surface[0][lost]
    not_actually_lost = at_surface[0][~lost]
    lost_electrons += len(actually_lost)
    electron_positions[not_actually_lost,2] = 0
    electron_positions[actually_lost,2] = np.NaN

diff_coeff = 36 # cm^2 / s
# Convert to um^2/ns
diff_coeff = diff_coeff * 1e8 / 1e9
time_step = 0.1 # ns
sigma = np.sqrt(2 * diff_coeff * time_step)
def execute_time_step():
    global electron_positions
    # Update the pixel location values for diffusion
    displacement_vals = np.random.multivariate_normal([0, 0, 0], [[sigma / 3, 0, 0], [0, sigma / 3, 0], [0, 0, sigma / 3]],
                                                      len(electron_positions))
    electron_positions += displacement_vals

i = 0
# Keep going until all electrons are collected or lost (i.e. all z vals are NaN)
while i < 2000 and not np.all(np.isnan(electron_positions[:,2])):
    collect_electrons()
    lose_electrons()
    execute_time_step()
    i += 1

central_pix_frac = collected_electrons[5,5] / np.sum(collected_electrons)
time_taken = i * time_step
print('Central Pixel Fraction:', format(central_pix_frac, '.2f'))
print('Time Taken:', format(time_taken, '.2f'), 'ns')
print('Quantum Efficiency:', format(np.sum(collected_electrons) / n_photons, '.3f'))

Central Pixel Fraction: 0.70
Time Taken: 68.00 ns
Quantum Efficiency: 0.902
